# Tạo Iceberg Tables cho Hệ Thống Phân Tích Tuyển Sinh

Notebook này sẽ tạo các bảng Iceberg trong Silver layer với Nessie catalog để quản lý metadata.

## 1. Import Libraries và Khởi tạo Spark Session

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import os

# Set AWS environment variables for MinIO
os.environ['AWS_REGION'] = 'us-east-1'
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'admin123'

# Khởi tạo Spark Session với Iceberg và Nessie catalog
spark = (
    SparkSession.builder.appName("Create_Silver_Iceberg_Tables")
    .master("spark://spark-master:7077")  # Kết nối tới Spark Cluster
    .config("spark.executor.memory", "2g")
    .config("spark.executor.cores", "2")
    # ===== Iceberg Catalog qua Nessie =====
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v2")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://silver/")
    .config("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    # ===== Cấu hình MinIO (S3-compatible) =====
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key-id", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-access-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .config("spark.sql.catalog.nessie.s3.region", "us-east-1")
    # ===== Spark + Hadoop S3 connector =====
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.region", "us-east-1")
    # Propagate environment variables to executors
    .config("spark.executorEnv.AWS_REGION", "us-east-1")
    .config("spark.executorEnv.AWS_ACCESS_KEY_ID", "admin")
    .config("spark.executorEnv.AWS_SECRET_ACCESS_KEY", "admin123")
    # ===== Sử dụng JAR files local (đã copy vào container) =====
    .config("spark.jars", "/opt/spark/jars/hadoop-aws-3.3.4.jar,/opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")
print("Spark Session da duoc khoi tao voi Nessie catalog!")
print(f"Spark Master: {spark.sparkContext.master}")
print(f"Application ID: {spark.sparkContext.applicationId}")

25/12/02 13:29:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark Session da duoc khoi tao voi Nessie catalog!
Spark Master: spark://spark-master:7077
Application ID: app-20251202132912-0000


## 2. Tạo Database/Namespace trong Nessie

In [2]:
# Tạo database silver nếu chưa tồn tại
spark.sql("CREATE DATABASE IF NOT EXISTS nessie.silver_tables")
spark.sql("USE nessie.silver_tables")

print(" Database 'silver' đã được tạo và đang sử dụng!")

 Database 'silver' đã được tạo và đang sử dụng!


## 3. Tạo Tables theo Schema

### 3.1. Table - School (Trường Đại Học)

In [3]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.silver_tables.school (
    schoolId STRING,
    schoolName STRING,
    province STRING,
    created_at TIMESTAMP,
    updated_at TIMESTAMP
) USING iceberg
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print(" Table 'school' đã được tạo!")

 Table 'school' đã được tạo!


### 3.2. Table - Major (Ngành Học)

In [4]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.silver_tables.major (
    majorId STRING,
    majorName STRING,
    created_at TIMESTAMP,
    updated_at TIMESTAMP
) USING iceberg
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print(" Table 'major' đã được tạo!")

 Table 'major' đã được tạo!


### 3.3. Table - SubjectGroup (Nhóm Môn/Khối Thi)

In [5]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.silver_tables.subject_group (
    subjectGroupId INT,
    subjectGroupName STRING,
    subjectCombination STRING,
    created_at TIMESTAMP,
    updated_at TIMESTAMP
) USING iceberg
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print(" Table 'subject_group' đã được tạo!")

 Table 'subject_group' đã được tạo!


### 3.4. Table - SelectionMethod (Phương Thức Xét Tuyển)

In [6]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.silver_tables.selection_method (
    selectionMethodId INT,
    selectionMethodName STRING,
    created_at TIMESTAMP,
    updated_at TIMESTAMP
) USING iceberg
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print(" Table 'selection_method' đã được tạo!")

 Table 'selection_method' đã được tạo!


### 3.5. Table - Benchmark (Điểm Chuẩn)

In [7]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.silver_tables.benchmark (
    benchmarkId INT,
    schoolId STRING,
    majorId STRING,
    subjectGroupId INT,
    selectionMethodId INT,
    gradingScaleId INT,
    year INT,
    score DOUBLE,
    created_at TIMESTAMP,
    updated_at TIMESTAMP
) USING iceberg
PARTITIONED BY (year)
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print(" Table 'benchmark' đã được tạo!")

 Table 'benchmark' đã được tạo!


### 3.6. Table - StudentScores (Điểm Thi Thí Sinh)

In [8]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.silver_tables.student_scores (
    studentId STRING,
    regionId STRING,
    year INT,
    scores MAP<STRING, DOUBLE>,
    created_at TIMESTAMP,
    updated_at TIMESTAMP
) USING iceberg
PARTITIONED BY (year)
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print(" Table 'student_scores' đã được tạo!")

 Table 'student_scores' đã được tạo!


### 3.7. Table - Region (Khu Vực)

In [9]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.silver_tables.region (
    regionId STRING,
    regionName STRING,
    created_at TIMESTAMP,
    updated_at TIMESTAMP
) USING iceberg
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print(" Table 'region' đã được tạo!")

 Table 'region' đã được tạo!


### 3.8. Table - Subject (Môn Học)

In [10]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.silver_tables.subject (
    subjectId INT,
    subjectName STRING,
    created_at TIMESTAMP,
    updated_at TIMESTAMP
) USING iceberg
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print(" Table 'subject' đã được tạo!")

 Table 'subject' đã được tạo!


### 3.8. Table - Grading Scale ( thang điểm)

In [11]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.silver_tables.grading_scale (
    gradingScaleId INT,
    value FLOAT,
    description STRING,
    created_at TIMESTAMP,
    updated_at TIMESTAMP
) USING iceberg
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print(" Table 'grading_scale' đã được tạo!")


 Table 'grading_scale' đã được tạo!


### 3.9. Table - Article (Bài viết, video TikTok)

In [12]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.silver_tables.article (
    articleID INT,
    title STRING,
    description STRING,
    author STRING,
    url STRING,
    timePublish TIMESTAMP,
    likeCount INT,
    commentCount INT,
    shareCount INT,
    type STRING,
    created_at TIMESTAMP,
    updated_at TIMESTAMP
) USING iceberg
PARTITIONED BY (days(timePublish))
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print(" Table 'article' đã được tạo!")

 Table 'article' đã được tạo!


### 3.10. Table - Comment

In [13]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.silver_tables.comment (
    commentID INT,
    articleID INT,
    name STRING,
    tagName STRING,
    urlUser STRING,
    comment STRING,
    commentTime TIMESTAMP,
    commentLike INT,
    levelComment INT,
    replyTo STRING,
    numberOfReply INT,
    created_at TIMESTAMP,
    updated_at TIMESTAMP
) USING iceberg
PARTITIONED BY (days(commentTime), articleID)
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print(" Table 'comment' đã được tạo!")

 Table 'comment' đã được tạo!


## 4. Kiểm Tra Các Tables Đã Tạo

In [14]:
# Liệt kê tất cả các tables trong database silver
tables = spark.sql("SHOW TABLES IN nessie.silver_tables")
tables.show(truncate=False)

+-------------+----------------+-----------+
|namespace    |tableName       |isTemporary|
+-------------+----------------+-----------+
|silver_tables|article         |false      |
|silver_tables|benchmark       |false      |
|silver_tables|comment         |false      |
|silver_tables|grading_scale   |false      |
|silver_tables|major           |false      |
|silver_tables|region          |false      |
|silver_tables|school          |false      |
|silver_tables|selection_method|false      |
|silver_tables|student_scores  |false      |
|silver_tables|subject         |false      |
|silver_tables|subject_group   |false      |
+-------------+----------------+-----------+



## 5. Dừng Spark Session

In [15]:
# Dừng Spark Session để giải phóng resources
spark.stop()
print(" Spark Session đã được dừng!")

 Spark Session đã được dừng!
